In [3]:
# !pip install redis


In [4]:
import redis

In [5]:
r = redis.Redis(
    host='redis-11993.c228.us-central1-1.gce.redns.redis-cloud.com',
    port=11993,
    decode_responses=True,
    username="default",
    password="ObIEQ5XPlF8xNDac7U7yiN6gtBUFVVUp",
)

In [6]:
import redis
import pandas as pd
import json

In [9]:
def save_df_to_redis(df, key):
    # Convert DataFrame to JSON string
    json_data = df.to_json()
    # Save to Redis
    r.set(key, json_data)

def get_df_from_redis(key):
    json_data = r.get(key)
    if json_data is not None:
        # Convert JSON back to DataFrame
        return pd.read_json(json_data)
    return None

# Example usage:
# Create sample DataFrame
df = pd.read_excel('youglish_full.xlsx')

save_df_to_redis(df, 'my_dataframe')



   name  age city
0  John   25   NY
1  Jane   30   LA
2   Bob   35   SF


C:\Users\thuylt16\AppData\Local\Temp\ipykernel_11772\750524093.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_data)


In [ ]:
# For dictionaries/databases:
def save_dict_to_redis(dict_data, key):
    # Convert dict to JSON string
    json_data = json.dumps(dict_data)
    r.set(key, json_data)

def get_dict_from_redis(key):
    json_data = r.get(key)
    if json_data is not None:
        return json.loads(json_data)
    return None

# Example with dictionary:
data = {
    'users': {
        '1': {'name': 'John', 'age': 25},
        '2': {'name': 'Jane', 'age': 30}
    }
}

# Save dictionary
save_dict_to_redis(data, 'my_database')

# Get dictionary
retrieved_data = get_dict_from_redis('my_database')
print(retrieved_data)

# Set expiration time (optional)
r.expire('my_database', 3600)

In [12]:
import re
def save_df_to_redis(df, key):
    # Convert DataFrame to JSON string with orient='records' for better structure
    json_data = df.to_json(orient='records')
    # Save to Redis
    r.set(key, json_data)

def get_df_from_redis(key):
    json_data = r.get(key)
    if json_data is not None:
        # Convert JSON back to DataFrame
        return pd.read_json(json_data, orient='records')
    return None

def search_words_in_df(df, phrases):
    """
    Search phrases in DataFrame and return one random result
    Args:
        df: DataFrame containing the captions
        phrases: list of words or phrases to search
    Returns:
        DataFrame with one random match including search word
    """
    results = []
    for phrase in phrases:
        words = phrase.split()
        if len(words) > 1:
            pattern = r'\b' + r'\s+'.join(map(re.escape, words)) + r'\b'
        else:
            pattern = fr'\b{re.escape(phrase)}\b'
            
        matches = df[df['caption'].str.contains(pattern, case=False, na=False, regex=True)]
        if not matches.empty:
            # Get one random row and add the search word
            random_match = matches.sample(n=1)[['video_id','video_title', 'start_second','end_second', 'caption']]
            random_match['word'] = phrase  # Add the search phrase as a new column
            results.append(random_match)
    
    if results:
        return pd.concat(results).reset_index(drop=True)
    return pd.DataFrame()



# 2. Subsequent searches:
def search_words(words):
    # Get full dataset from Redis
    df = get_df_from_redis('youglish_data')
    if df is not None:
        return search_words_in_df(df, words)
    return pd.DataFrame()

# # Example search
# phrases_to_search = ['folk music', 'go', 'beat']
# results = search_words(phrases_to_search)
# print(results)

      video_id                                        video_title  \
0  j7AzDpkq8m8                     Does Free Will Actually Exist?   
1  9axG7EhGNmI  Holly Herndon and Jace Clayton in conversation...   
2  AtfvZ_KOiZY  Seth Godin on Why “Finding Your Calling” Is Co...   

   start_second  end_second  \
0           152       162.0   
1          1063      1069.0   
2          1383      1390.0   

                                             caption        word  
0  to frighten the bear off with disco lights and...  folk music  
1  And, of course I have to let go sometimes and ...          go  
2  Don't turn the board over and don't beat yours...        beat  


C:\Users\thuylt16\AppData\Local\Temp\ipykernel_32928\2310443879.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_data, orient='records')


In [15]:
# Example usage:
# 1. First time: Load from Excel and save to Redis
df = pd.read_csv('youglish_full2.csv')
save_df_to_redis(df, 'youglish_data')

In [17]:
words = pd.read_csv('error_words.csv')
words = words['for instance'].tolist()
words



['in favour (of)',
 'in general',
 'look something up',
 'point something out',
 'rip something up',
 '<div><div><div><div><div></div></div></div></div></div><div><div><div><div><div><div></div></div></div></div></div><div>rub out<br></div></div>',
 'write something down',
 'begin',
 'correct',
 'division',
 'begun',
 'beginner',
 'bravery',
 'incorrect',
 'education',
 'instruct',
 'memory',
 'memorize',
 'refer someone to something/someone',
 'reference',
 'silence',
 'silently',
 'simplify',
 'simplicity',
 'talented at',
 'cheat at/in',
 'confuse sth with',
 'continue with',
 'cope with',
 'help (sb) with',
 'know about',
 'learn about',
 'an opinion about/of',
 'a question about',
 'boyfriend',
 'close',
 'cool',
 'couple',
 'defend',
 'divorced',
 'generous',
 'girlfriend',
 'guest',
 'independent',
 'loving',
 'loyal',
 'neighbourhood',
 'ordinary',
 'patient',
 'recognize',
 'relation',
 'respect',
 'single',
 'trust',
 'bring someone up',
 'get on with someone',
 'go out',
 'l

In [18]:
df = get_df_from_redis('youglish_data')
df

C:\Users\thuylt16\AppData\Local\Temp\ipykernel_32928\2310443879.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_data, orient='records')


,video_id,video_title,start_second,end_second,caption
0,5MgBikgcWnY,The first 20 hours -- how to learn anything | ...,7,13.0,Hi everyone.
1,5MgBikgcWnY,The first 20 hours -- how to learn anything | ...,9,18.0,"Two year ago, my life changed forever."
2,5MgBikgcWnY,The first 20 hours -- how to learn anything | ...,14,21.0,My wife Kelsey and I
3,5MgBikgcWnY,The first 20 hours -- how to learn anything | ...,17,25.0,welcomed our daughter Lela into the world.
4,5MgBikgcWnY,The first 20 hours -- how to learn anything | ...,21,29.0,"Now, becoming a parent is an amazing experience."
...,...,...,...,...,...
47170,7xeFP0SEDdc,Stroll Through the Playlist (a Biology Review),2448,2456.0,Don't forget we also have a video with study s...
47171,7xeFP0SEDdc,Stroll Through the Playlist (a Biology Review),2452,2461.0,and we have helpful GIF animations and comics ...
47172,7xeFP0SEDdc,Stroll Through the Playlist (a Biology Review),2457,2467.0,And…if you are studying for something big… it ...
47173,7xeFP0SEDdc,Stroll Through the Playlist (a Biology Review),2463,2469.0,feel confident about your learning.


In [19]:
results = search_words(words)
results

C:\Users\thuylt16\AppData\Local\Temp\ipykernel_32928\2310443879.py:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_data, orient='records')


,video_id,video_title,start_second,end_second,caption,word
0,9JUzxdeYRZc,Timothy Snyder: The Making of Modern Ukraine. ...,1478,1485.0,"and in general, they wanted to have more land ...",in general
1,ZK5VUuxGYr0,Minimal bullet journal setup » for productivit...,201,210.0,I begin by splitting each page into three colu...,begin
2,YjBjrlnas40,President Obama Speaks at the Concussion Summit,3090,3095.0,"Cheryl, correct?",correct
3,7xeFP0SEDdc,Stroll Through the Playlist (a Biology Review),664,673.0,That's really important later on when we talk ...,division
4,9VF7kffCgJU,Drawings that show the beauty and fragility of...,336,343.0,which have already begun to drown,begun
...,...,...,...,...,...,...
532,CoamGIfXvhU,"Napoleon's Marshals: Bessières, Macdonald, Mas...",627,637.0,"In the Revolutionary Wars, he won a reputation...",intelligent
533,td7Dcsco-WY,Obama on the state of the world: the extended ...,2095,2106.0,"resolution when nobody denies, including our i...",intelligence
534,td7Dcsco-WY,Obama on the state of the world: the extended ...,98,111.0,a pretty good job with that. It wasn't perfect...,perfect
535,Grv1RJkdyqI,I Asked Bill Gates What's The Next Crisis?,190,199.0,human-caused nuclear war that we needed to be ...,prepared


In [21]:
len(words)


722